<a href="https://colab.research.google.com/github/AleksandreBakhtadze/ML-abakh22-facial-expression-recognition/blob/main/facial_expression_train4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [2]:
!pip install -q kaggle
!pip install -q wandb
!pip install -q torchmetrics
!pip install -q albumentations

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 962.5/962.5 kB 27.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 1.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 122.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 92.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 58.9 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.2 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 5.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 15.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 7.5 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 5.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 26.6 MB/s eta 0:00:00


In [3]:
import wandb
wandb.login()

!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/
!chmod 600 ~/.kaggle/kaggle.json
!kaggle competitions download -c challenges-in-representation-learning-facial-expression-recognition-challenge
!unzip -q challenges-in-representation-learning-facial-expression-recognition-challenge.zip

<IPython.core.display.Javascript object>

wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: abakh22 (abakh22-free-university-of-tbilisi-) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


 92% 262M/285M [00:00<00:00, 290MB/s]
100% 285M/285M [00:00<00:00, 316MB/s]


In [18]:
import os
import torch
import torchvision
from torchvision import datasets, transforms
from torch.utils.data import DataLoader, Subset
import torch.nn as nn
import torch.optim as optim
from torch.optim.lr_scheduler import ReduceLROnPlateau
from torchmetrics.classification import MulticlassConfusionMatrix, MulticlassPrecision, MulticlassRecall, MulticlassF1Score
import matplotlib.pyplot as plt
import seaborn as sns
from torchsummary import summary
import numpy as np
from PIL import Image
import albumentations as A
import torch.nn.functional as F
from albumentations.pytorch import ToTensorV2

In [19]:
import pandas as pd

# Load the CSV
df = pd.read_csv("train.csv")

# Example row
print(df.head())

# Convert the pixel values to image tensors
def process_row(row):
    pixels = np.array([int(p) for p in row['pixels'].split()], dtype=np.uint8).reshape(48, 48)
    img = Image.fromarray(pixels)
    return img, int(row['emotion'])

images, labels = zip(*[process_row(row) for _, row in df.iterrows()])

   emotion                                             pixels
0        0  70 80 82 72 58 58 60 63 54 58 60 48 89 115 121...
1        0  151 150 147 155 148 133 111 140 170 174 182 15...
2        2  231 212 156 164 174 138 161 173 182 200 106 38...
3        4  24 32 36 30 32 23 19 20 30 41 21 22 32 34 21 1...
4        6  4 0 0 0 0 0 0 0 0 0 0 0 3 15 23 28 48 50 58 84...


In [20]:
# Define separate transforms for training and validation
# Enhanced transforms
train_transform = A.Compose([
    A.HorizontalFlip(p=0.5),
    A.ShiftScaleRotate(shift_limit=0.1, scale_limit=0.1, rotate_limit=15, p=0.5),
    A.RandomBrightnessContrast(brightness_limit=0.2, contrast_limit=0.2, p=0.5),
    A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
    A.CoarseDropout(max_holes=8, max_height=8, max_width=8, fill_value=0, p=0.3),
    A.Normalize(mean=[0.485], std=[0.229]),  # Using standard ImageNet stats
    ToTensorV2()
])

val_transform = A.Compose([
    A.Normalize(mean=[0.485], std=[0.229]),
    ToTensorV2()
])
class FERDataset(torch.utils.data.Dataset):
    def __init__(self, images, labels, transform=None):
        self.images = images
        self.labels = labels
        self.transform = transform

    def __len__(self):
        return len(self.labels)

    def __getitem__(self, index):
        img = self.images[index]  # Fixed: Changed 'idx' to 'index'
        img = np.array(self.images[index])
        label = self.labels[index]
        if self.transform:
            augmented = self.transform(image=img)
            img = augmented['image']
        return img, label

# Split into train and val
from sklearn.model_selection import train_test_split
from collections import Counter
train_imgs, val_imgs, train_labels, val_labels = train_test_split(images, labels, test_size=0.1, stratify=labels, random_state=42)

# Add WeightedRandomSampler for class imbalance
class_counts = Counter(train_labels)
num_samples = len(train_labels)
class_weights = {i: num_samples / (len(class_counts) * count) for i, count in class_counts.items()}
sample_weights = [class_weights[label] for label in train_labels]
sampler = torch.utils.data.WeightedRandomSampler(sample_weights, num_samples)

train_dataset = FERDataset(train_imgs, train_labels, train_transform)
val_dataset = FERDataset(val_imgs, val_labels, val_transform)

train_loader = DataLoader(train_dataset, batch_size=128, sampler=sampler)  # Use sampler
val_loader = DataLoader(val_dataset, batch_size=32)  # Smaller batch size for validation

/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)
<ipython-input-20-7722ad1ccef0>:7: UserWarning: Argument(s) 'var_limit' are not valid for transform GaussNoise
  A.GaussNoise(var_limit=(10.0, 50.0), p=0.3),
<ipython-input-20-7722ad1ccef0>:8: UserWarning: Argument(s) 'max_holes, max_height, max_width, fill_value' are not valid for transform CoarseDropout
  A.CoarseDropout(max_holes=8, max_height=8, max_width=8, fill_value=0, p=0.3),


In [21]:
class BasicFERModel(nn.Module):
    def __init__(self, num_classes=7):
        super(BasicFERModel, self).__init__()

        self.features = nn.Sequential(
            nn.Conv2d(1, 32, kernel_size=3, padding=1),
            nn.BatchNorm2d(32),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25),

            nn.Conv2d(32, 64, kernel_size=3, padding=1),
            nn.BatchNorm2d(64),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25),

            nn.Conv2d(64, 128, kernel_size=3, padding=1),
            nn.BatchNorm2d(128),
            nn.ReLU(inplace=True),
            nn.MaxPool2d(kernel_size=2, stride=2),
            nn.Dropout(0.25)
        )

        self.classifier = nn.Sequential(
            nn.Linear(128 * 6 * 6, 256),
            nn.BatchNorm1d(256),
            nn.ReLU(inplace=True),
            nn.Dropout(0.5),
            nn.Linear(256, num_classes)
        )

    def forward(self, x):
        x = self.features(x)
        x = x.view(x.size(0), -1)
        x = self.classifier(x)
        return x

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = BasicFERModel().to(device)
summary(model, (1, 48, 48))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1           [-1, 32, 48, 48]             320
       BatchNorm2d-2           [-1, 32, 48, 48]              64
              ReLU-3           [-1, 32, 48, 48]               0
         MaxPool2d-4           [-1, 32, 24, 24]               0
           Dropout-5           [-1, 32, 24, 24]               0
            Conv2d-6           [-1, 64, 24, 24]          18,496
       BatchNorm2d-7           [-1, 64, 24, 24]             128
              ReLU-8           [-1, 64, 24, 24]               0
         MaxPool2d-9           [-1, 64, 12, 12]               0
          Dropout-10           [-1, 64, 12, 12]               0
           Conv2d-11          [-1, 128, 12, 12]          73,856
      BatchNorm2d-12          [-1, 128, 12, 12]             256
             ReLU-13          [-1, 128, 12, 12]               0
        MaxPool2d-14            [-1, 12

In [22]:
# Create a small subset of the training set to test overfitting
small_indices = list(range(64))  # Tiny sample size
small_dataset = torch.utils.data.Subset(train_dataset, small_indices)
small_loader = DataLoader(small_dataset, batch_size=16, shuffle=True)

def test_overfitting():
    class TinyModel(nn.Module):
        def __init__(self, num_classes=7):
            super(TinyModel, self).__init__()
            self.conv = nn.Conv2d(1, 8, kernel_size=3, padding=1)
            self.pool = nn.MaxPool2d(2)
            self.fc = nn.Linear(8*24*24, num_classes)

        def forward(self, x):
            x = self.pool(F.relu(self.conv(x)))
            x = x.view(x.size(0), -1)
            x = self.fc(x)
            return x

    wandb.init(project="FER-CNN", name="overfit_test_v4")

    model = TinyModel().to(device)
    optimizer = optim.SGD(model.parameters(), lr=0.01)
    criterion = nn.CrossEntropyLoss()

    print("Training on small dataset to check overfitting...")
    for epoch in range(50):
        model.train()
        total_loss, correct = 0, 0

        for inputs, labels in small_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            optimizer.step()

            total_loss += loss.item()
            correct += (outputs.argmax(1) == labels).sum().item()

        acc = correct / len(small_dataset)
        wandb.log({
            "overfit_epoch": epoch + 1,
            "overfit_loss": total_loss,
            "overfit_acc": acc
        })

        print(f"Epoch {epoch+1} - Loss: {total_loss:.4f}, Acc: {acc:.4f}")
        if acc >= 0.95:
            print("✅ Model successfully overfit small dataset")
            break

    wandb.finish()

test_overfitting()


Training on small dataset to check overfitting...
Epoch 1 - Loss: 9.4222, Acc: 0.1250
Epoch 2 - Loss: 8.5299, Acc: 0.2344
Epoch 3 - Loss: 7.8905, Acc: 0.3281
Epoch 4 - Loss: 6.8035, Acc: 0.3125
Epoch 5 - Loss: 8.1665, Acc: 0.2656
Epoch 6 - Loss: 8.3051, Acc: 0.3438
Epoch 7 - Loss: 9.3703, Acc: 0.2188
Epoch 8 - Loss: 6.8226, Acc: 0.2969
Epoch 9 - Loss: 7.2416, Acc: 0.2812
Epoch 10 - Loss: 7.8369, Acc: 0.3125
Epoch 11 - Loss: 6.4243, Acc: 0.4062
Epoch 12 - Loss: 6.3565, Acc: 0.3281
Epoch 13 - Loss: 7.1413, Acc: 0.2656
Epoch 14 - Loss: 6.7042, Acc: 0.3594
Epoch 15 - Loss: 6.5307, Acc: 0.4062
Epoch 16 - Loss: 6.5642, Acc: 0.4219
Epoch 17 - Loss: 5.8726, Acc: 0.5000
Epoch 18 - Loss: 6.0906, Acc: 0.3906
Epoch 19 - Loss: 5.9835, Acc: 0.3906
Epoch 20 - Loss: 5.7056, Acc: 0.4375
Epoch 21 - Loss: 5.3460, Acc: 0.4688
Epoch 22 - Loss: 5.3666, Acc: 0.4844
Epoch 23 - Loss: 5.8345, Acc: 0.4219
Epoch 24 - Loss: 5.3596, Acc: 0.4219
Epoch 25 - Loss: 7.6101, Acc: 0.3750
Epoch 26 - Loss: 5.3295, Acc: 0.56

overfit_acc,▁▂▄▃▃▂▃▃▃▄▄▅▅▆▅▅▆▆▅▅▇▆▆▅▇▇▆█▆▆▆██▇▇▇█▆▇▇
overfit_epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▆▆▆▆▆▆▇▇▇▇▇▇███
overfit_loss,█▇▆▅▆▅▅▆▄▄▄▄▄▃▄▃▃▃▃▃▃▃▂▃▃▃▃▂▃▂▃▂▂▂▂▂▁▂▂▁
overfit_acc,0.60938
overfit_epoch,50
overfit_loss,4.00486


In [30]:
from sklearn.metrics import confusion_matrix, classification_report

def train_model(model, config):
    wandb.init(project="FER-CNN", config=config)

    # Transforms
    train_transform = A.Compose([
        A.HorizontalFlip(p=0.3),
        A.ShiftScaleRotate(shift_limit=0.05, scale_limit=0.05, rotate_limit=10, p=0.3),
        A.RandomBrightnessContrast(brightness_limit=0.1, contrast_limit=0.1, p=0.3),
        A.Normalize(mean=[0.5], std=[0.5]),
        ToTensorV2()
    ])

    val_transform = A.Compose([
        A.Normalize(mean=[0.5], std=[0.5]),
        ToTensorV2()
    ])

    # Recreate datasets
    train_dataset = FERDataset(train_imgs, train_labels, train_transform)
    val_dataset = FERDataset(val_imgs, val_labels, val_transform)

    # Handle class imbalance
    class_counts = Counter(train_labels)
    num_samples = len(train_labels)
    class_weights = {i: num_samples / (len(class_counts) * count) for i, count in class_counts.items()}
    sample_weights = [class_weights[label] for label in train_labels]
    sampler = torch.utils.data.WeightedRandomSampler(sample_weights, num_samples)

    train_loader = DataLoader(train_dataset, batch_size=config['batch_size'], sampler=sampler)
    val_loader = DataLoader(val_dataset, batch_size=config['batch_size'])

    model = model.to(device)
    optimizer = optim.AdamW(model.parameters(), lr=config['lr'], weight_decay=config['weight_decay'])
    scheduler = optim.lr_scheduler.CosineAnnealingWarmRestarts(optimizer, T_0=10, T_mult=1, eta_min=1e-6)

    # Focal Loss
    weights_tensor = torch.tensor([class_weights[i] for i in range(len(class_weights))]).float().to(device)
    criterion = FocalLoss(gamma=2.0, weight=weights_tensor)

    # Metrics
    metrics = {
        'conf_matrix': MulticlassConfusionMatrix(num_classes=7).to(device),
        'f1': MulticlassF1Score(num_classes=7, average='macro').to(device),
        'precision': MulticlassPrecision(num_classes=7, average='macro').to(device),
        'recall': MulticlassRecall(num_classes=7, average='macro').to(device)
    }

    best_val_acc = 0.0
    patience_counter = 0

    for epoch in range(config['epochs']):
        model.train()
        train_loss, train_correct = 0.0, 0

        for inputs, labels in train_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            optimizer.zero_grad()
            outputs = model(inputs)
            loss = criterion(outputs, labels)
            loss.backward()
            torch.nn.utils.clip_grad_norm_(model.parameters(), 1.0)
            optimizer.step()

            train_loss += loss.item()
            train_correct += (outputs.argmax(1) == labels).sum().item()

        train_acc = train_correct / len(train_loader.dataset)

        # Validation
        model.eval()
        val_loss, val_correct = 0.0, 0
        all_preds, all_labels = [], []

        for metric in metrics.values():
            metric.reset()

        with torch.no_grad():
            for inputs, labels in val_loader:
                inputs, labels = inputs.to(device), labels.to(device)
                outputs = model(inputs)
                loss = criterion(outputs, labels)
                val_loss += loss.item()

                preds = outputs.argmax(1)
                val_correct += (preds == labels).sum().item()

                all_preds.extend(preds.cpu().numpy())
                all_labels.extend(labels.cpu().numpy())

                for metric in metrics.values():
                    metric.update(preds, labels)

        val_acc = val_correct / len(val_loader.dataset)
        scheduler.step()

        # Compute metrics safely
        metric_results = {}
        for name, metric in metrics.items():
            result = metric.compute().detach().cpu()
            if result.numel() == 1:
                metric_results[name] = result.item()
            else:
                metric_results[name] = wandb.Histogram(result.numpy())

        # Log to wandb
        wandb.log({
            'epoch': epoch + 1,
            'train_loss': train_loss / len(train_loader),
            'train_acc': train_acc,
            'val_loss': val_loss / len(val_loader),
            'val_acc': val_acc,
            'lr': optimizer.param_groups[0]['lr'],
            **metric_results
        })

        print(f"Epoch {epoch+1}/{config['epochs']} - Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

        # Early stopping
        if val_acc > best_val_acc:
            best_val_acc = val_acc
            patience_counter = 0
            torch.save(model.state_dict(), 'best_model.pth')
        else:
            patience_counter += 1
            if patience_counter >= config['patience']:
                print(f"Early stopping at epoch {epoch+1}")
                break

    # Final evaluation and confusion matrix
    model.load_state_dict(torch.load('best_model.pth'))
    model.eval()
    all_preds, all_labels = [], []

    with torch.no_grad():
        for inputs, labels in val_loader:
            inputs, labels = inputs.to(device), labels.to(device)
            outputs = model(inputs)
            preds = outputs.argmax(1)
            all_preds.extend(preds.cpu().numpy())
            all_labels.extend(labels.cpu().numpy())

    cm = confusion_matrix(all_labels, all_preds)
    fig, ax = plt.subplots(figsize=(10, 8))
    sns.heatmap(cm, annot=True, fmt='d', cmap='Blues', ax=ax)
    ax.set_xlabel("Predicted")
    ax.set_ylabel("True")
    ax.set_title("Confusion Matrix")
    wandb.log({"confusion_matrix": wandb.Image(fig)})
    plt.close(fig)

    # Classification report
    report = classification_report(
        all_labels, all_preds,
        target_names=['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral'],
        digits=4
    )
    print(report)
    wandb.run.summary["classification_report"] = report

    # Save model as artifact
    artifact = wandb.Artifact(f"{config['model_name']}_model", type="model")
    artifact.add_file("best_model.pth")
    wandb.log_artifact(artifact)
    wandb.finish()


# Focal Loss implementation
class FocalLoss(nn.Module):
    def __init__(self, gamma=2.0, weight=None):
        super(FocalLoss, self).__init__()
        self.gamma = gamma
        self.weight = weight
        self.ce = nn.CrossEntropyLoss(weight=weight)

    def forward(self, input, target):
        logpt = -self.ce(input, target)
        pt = torch.exp(logpt)
        loss = -((1 - pt) ** self.gamma) * logpt
        return loss.mean()

In [32]:
# Experiment 1: Basic CNN
config = {
    'model_name': 'basic_cnn',
    'lr': 0.001,
    'batch_size': 64,
    'epochs': 50,
    'weight_decay': 1e-4,
    'patience': 10
}
train_model(BasicFERModel(), config)

# Experiment 2: Residual Network
class ResidualBlock(nn.Module):
    def __init__(self, in_channels, out_channels, stride=1):
        super(ResidualBlock, self).__init__()
        self.conv1 = nn.Conv2d(in_channels, out_channels, kernel_size=3, stride=stride, padding=1)
        self.bn1 = nn.BatchNorm2d(out_channels)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = nn.Conv2d(out_channels, out_channels, kernel_size=3, padding=1)
        self.bn2 = nn.BatchNorm2d(out_channels)

        self.shortcut = nn.Sequential()
        if stride != 1 or in_channels != out_channels:
            self.shortcut = nn.Sequential(
                nn.Conv2d(in_channels, out_channels, kernel_size=1, stride=stride),
                nn.BatchNorm2d(out_channels)
            )

    def forward(self, x):
        residual = self.shortcut(x)
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.conv2(out)
        out = self.bn2(out)
        out += residual
        out = self.relu(out)
        return out

class ResNetFER(nn.Module):
    def __init__(self, num_classes=7):
        super(ResNetFER, self).__init__()
        self.in_channels = 32
        self.conv1 = nn.Conv2d(1, 32, kernel_size=3, padding=1)
        self.bn1 = nn.BatchNorm2d(32)
        self.relu = nn.ReLU(inplace=True)
        self.layer1 = self.make_layer(32, 2, stride=1)
        self.layer2 = self.make_layer(64, 2, stride=2)
        self.layer3 = self.make_layer(128, 2, stride=2)
        self.avg_pool = nn.AdaptiveAvgPool2d((1, 1))
        self.fc = nn.Linear(128, num_classes)

    def make_layer(self, out_channels, blocks, stride):
        layers = []
        layers.append(ResidualBlock(self.in_channels, out_channels, stride))
        self.in_channels = out_channels
        for _ in range(1, blocks):
            layers.append(ResidualBlock(out_channels, out_channels))
        return nn.Sequential(*layers)

    def forward(self, x):
        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)
        out = self.layer1(out)
        out = self.layer2(out)
        out = self.layer3(out)
        out = self.avg_pool(out)
        out = out.view(out.size(0), -1)
        out = self.fc(out)
        return out

# Train ResNet
config_resnet = {
    'model_name': 'resnet',
    'lr': 0.001,
    'batch_size': 64,
    'epochs': 50,
    'weight_decay': 1e-4,
    'patience': 10
}
train_model(ResNetFER(), config_resnet)

# Experiment 3: Vision Transformer (small version)
class ViTFER(nn.Module):
    def __init__(self, num_classes=7, image_size=48, patch_size=8, dim=64, depth=4, heads=4):
        super(ViTFER, self).__init__()
        num_patches = (image_size // patch_size) ** 2
        patch_dim = 1 * patch_size ** 2

        self.patch_size = patch_size
        self.pos_embedding = nn.Parameter(torch.randn(1, num_patches + 1, dim))
        self.patch_to_embedding = nn.Linear(patch_dim, dim)
        self.cls_token = nn.Parameter(torch.randn(1, 1, dim))

        self.transformer = nn.TransformerEncoder(
            nn.TransformerEncoderLayer(d_model=dim, nhead=heads),
            num_layers=depth
        )

        self.to_cls_token = nn.Identity()
        self.mlp_head = nn.Sequential(
            nn.Linear(dim, dim // 2),
            nn.ReLU(),
            nn.Linear(dim // 2, num_classes)
        )

    def forward(self, img):
        p = self.patch_size
        bs, c, h, w = img.shape
        img = img.unfold(2, p, p).unfold(3, p, p)
        img = img.contiguous().view(bs, c, -1, p, p)
        img = img.permute(0, 2, 3, 4, 1)
        img = img.contiguous().view(bs, -1, p * p * c)

        x = self.patch_to_embedding(img)
        cls_tokens = self.cls_token.expand(bs, -1, -1)
        x = torch.cat((cls_tokens, x), dim=1)
        x += self.pos_embedding
        x = self.transformer(x)
        x = self.to_cls_token(x[:, 0])
        return self.mlp_head(x)


# Train ViT
config_vit = {
    'model_name': 'vit',
    'lr': 0.0005,  # Lower LR for ViT
    'batch_size': 32,  # Smaller batch size due to memory
    'epochs': 50,
    'weight_decay': 1e-4,
    'patience': 10
}
train_model(ViTFER(), config_vit)

/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


Epoch 1/50 - Train Acc: 0.2562, Val Acc: 0.1456
Epoch 2/50 - Train Acc: 0.3535, Val Acc: 0.3163
Epoch 3/50 - Train Acc: 0.4126, Val Acc: 0.4009
Epoch 4/50 - Train Acc: 0.4348, Val Acc: 0.4026
Epoch 5/50 - Train Acc: 0.4625, Val Acc: 0.4330
Epoch 6/50 - Train Acc: 0.4800, Val Acc: 0.4573
Epoch 7/50 - Train Acc: 0.4949, Val Acc: 0.4716
Epoch 8/50 - Train Acc: 0.5031, Val Acc: 0.4772
Epoch 9/50 - Train Acc: 0.5051, Val Acc: 0.4859
Epoch 10/50 - Train Acc: 0.5105, Val Acc: 0.4866
Epoch 11/50 - Train Acc: 0.4926, Val Acc: 0.4678
Epoch 12/50 - Train Acc: 0.5002, Val Acc: 0.4876
Epoch 13/50 - Train Acc: 0.5086, Val Acc: 0.5110
Epoch 14/50 - Train Acc: 0.5221, Val Acc: 0.5127
Epoch 15/50 - Train Acc: 0.5322, Val Acc: 0.4998
Epoch 16/50 - Train Acc: 0.5392, Val Acc: 0.5155
Epoch 17/50 - Train Acc: 0.5505, Val Acc: 0.5246
Epoch 18/50 - Train Acc: 0.5538, Val Acc: 0.5326
Epoch 19/50 - Train Acc: 0.5581, Val Acc: 0.5402
Epoch 20/50 - Train Acc: 0.5602, Val Acc: 0.5392
Epoch 21/50 - Train Acc: 0.54

epoch,▁▁▁▁▂▂▂▂▂▂▃▃▃▃▃▄▄▄▄▄▅▅▅▅▅▅▆▆▆▆▆▇▇▇▇▇▇███
f1,▁▄▅▅▅▆▆▆▆▆▆▆▆▇▇▆▇▇▇▇▇▇█▇▇▇█▇▇▇██▇▇▇█████
lr,█▇▇▆▄▂▂▁██▆▄▃▂▂██▇▇▄▂▂▁██▇▆▄▃▂▁██▇▇▄▃▂▂█
precision,▁▃▄▄▄▄▄▅▅▅▆▅▅▅▆▆▆▅▆▆▆▆▇▇▇▆▆▇▇▇▇██▇▆▇█▇██
recall,▁▄▅▅▆▆▆▇▇▆▇▇▇▇▇█▇▇▇▇█████▇▇███████▇█████
train_acc,▁▃▄▄▅▆▆▆▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇▇▇▇▇█████▇▇█████
train_loss,█▅▄▄▃▃▂▂▂▃▂▂▂▂▂▂▂▂▂▂▂▁▁▁▁▂▂▁▁▁▁▁▁▁▁▁▁▁▁▁
val_acc,▁▄▅▅▆▆▆▆▆▆▇▇▇▇▇▇▇▇▇▇████▇▇███████▇██████
val_loss,█▄▃▃▂▂▂▂▂▂▂▂▁▁▁▁▁▂▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁▁
classification_report,precis...
epoch,50


/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


Epoch 1/50 - Train Acc: 0.1912, Val Acc: 0.0763
Epoch 2/50 - Train Acc: 0.3151, Val Acc: 0.2699
Epoch 3/50 - Train Acc: 0.4179, Val Acc: 0.3800
Epoch 4/50 - Train Acc: 0.4998, Val Acc: 0.4103
Epoch 5/50 - Train Acc: 0.5435, Val Acc: 0.5120
Epoch 6/50 - Train Acc: 0.5689, Val Acc: 0.4904
Epoch 7/50 - Train Acc: 0.5930, Val Acc: 0.5082
Epoch 8/50 - Train Acc: 0.6140, Val Acc: 0.5535
Epoch 9/50 - Train Acc: 0.6258, Val Acc: 0.5517
Epoch 10/50 - Train Acc: 0.6374, Val Acc: 0.5611
Epoch 11/50 - Train Acc: 0.5547, Val Acc: 0.4622
Epoch 12/50 - Train Acc: 0.5818, Val Acc: 0.4723
Epoch 13/50 - Train Acc: 0.6065, Val Acc: 0.4197
Epoch 14/50 - Train Acc: 0.6154, Val Acc: 0.5545
Epoch 15/50 - Train Acc: 0.6418, Val Acc: 0.5113
Epoch 16/50 - Train Acc: 0.6570, Val Acc: 0.5347
Epoch 17/50 - Train Acc: 0.6781, Val Acc: 0.5448
Epoch 18/50 - Train Acc: 0.6852, Val Acc: 0.6047
Epoch 19/50 - Train Acc: 0.6959, Val Acc: 0.6095
Epoch 20/50 - Train Acc: 0.7030, Val Acc: 0.6001
Epoch 21/50 - Train Acc: 0.61

...

/usr/local/lib/python3.11/dist-packages/torch/nn/modules/transformer.py:385: UserWarning: enable_nested_tensor is True, but self.use_nested_tensor is False because encoder_layer.self_attn.batch_first was not True(use batch_first for better inference performance)
  warnings.warn(


/usr/local/lib/python3.11/dist-packages/albumentations/core/validation.py:114: UserWarning: ShiftScaleRotate is a special case of Affine transform. Please use Affine transform instead.
  original_init(self, **validated_kwargs)


Epoch 1/50 - Train Acc: 0.1398, Val Acc: 0.0153
Epoch 2/50 - Train Acc: 0.1453, Val Acc: 0.0153
Epoch 3/50 - Train Acc: 0.1464, Val Acc: 0.0153
Epoch 4/50 - Train Acc: 0.1391, Val Acc: 0.0153
Epoch 5/50 - Train Acc: 0.1454, Val Acc: 0.0153
Epoch 6/50 - Train Acc: 0.1435, Val Acc: 0.0153
Epoch 7/50 - Train Acc: 0.1437, Val Acc: 0.0153
Epoch 8/50 - Train Acc: 0.1449, Val Acc: 0.0153
Epoch 9/50 - Train Acc: 0.1458, Val Acc: 0.0153
Epoch 10/50 - Train Acc: 0.1412, Val Acc: 0.0153
Epoch 11/50 - Train Acc: 0.1468, Val Acc: 0.0153
Early stopping at epoch 11
              precision    recall  f1-score   support

       Angry     0.0000    0.0000    0.0000       399
     Disgust     0.0153    1.0000    0.0302        44
        Fear     0.0000    0.0000    0.0000       410
       Happy     0.0000    0.0000    0.0000       722
         Sad     0.0000    0.0000    0.0000       483
    Surprise     0.0000    0.0000    0.0000       317
     Neutral     0.0000    0.0000    0.0000       496

    accur

/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/usr/local/lib/python3.11/dist-packages/sklearn/metrics/_classification.py:1565: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))


epoch,▁▂▂▃▄▅▅▆▇▇█
f1,▁▁▁▁▁▁▁▁▁▁▁
lr,█▇▇▆▄▃▂▂▁██
precision,▁▁▁▁▁▁▁▁▁▁▁
recall,▁▁▁▁▁▁▁▁▁▁▁
train_acc,▂▇█▁▇▅▅▆▇▃█
train_loss,▇▃▁█▂▅▄▅▂▆▂
val_acc,▁▁▁▁▁▁▁▁▁▁▁
val_loss,█▆█▁▆▄▅▅▄▄▅
classification_report,precis...
epoch,11
